## Calculate the production rate
---
1. Read information from Sheet "Initial"
2. Calculate production rates in each sub Sheet

* Be really really careful on unit, the current code is based on unit:
  1. thickness: mg/cm2;
  2. molar mass: g;
  3. x-section: mb;
  4. beam rate: pps;

In [8]:
import pdfplumber
import os
import re
import pandas as pd
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter
from openpyxl.styles import Font, PatternFill, Border, Alignment

### Read Sheet "Initial":
1. thickness in "mg/cm2";
2. molar mass in "g"
3. beam rate in "pps"

In [29]:
def ReadInitial(excel_path):
    sheets = pd.read_excel(excel_path, sheet_name=None)  # Load all sheets
    df = sheets["Initial"]
    
    # locate the target info
    target_col_index = df.columns.get_loc('Target')
    thick_row_idx = df[df.iloc[:, target_col_index + 2] == 'mg/cm2'].index[0]
    thick_tot = df.iloc[thick_row_idx, target_col_index + 1]
    mass_row_idx = df[df.iloc[:, target_col_index + 2] == 'g'].index[0]
    mass = df.iloc[mass_row_idx, target_col_index + 1]

    # locate the beam info
    beam_col_index = df.columns.get_loc('Beam')
    rate_row_idx = df[df.iloc[:, beam_col_index + 2] == 'pps'].index[0]
    rate = df.iloc[rate_row_idx, beam_col_index + 1]

    return thick_tot, mass, rate

### Calculate production rate in on Sheet

In [30]:
def Cal_Prodx_Rate(df, thick_tot, mass, rate):
    if "Prodx_Rate(pps)" not in df.columns:
        df["Prodx_Rate(pps)"] = -1.0
    for idx in range(len(df)):
        xsec = df.at[idx,'x-section(mb)']
        # Eqn: prodx_rate = xsec(m2) * unit_thick(g/m2) / mass(g) *rate (atom/s)
        # 1 mb = 1e-3 barn = 1e-3 * 1e-28 m2
        # 1 mg/cm2 = 10 g/m2
        df.at[idx,'Prodx_Rate(pps)'] = (xsec/1e3/1e28)*((thick_tot/5)*10)/mass*rate
    return df

### Update all sheets in Excel

In [35]:
def Write_Prodx_Rate(excel_path):
    thick_tot, mass, rate = ReadInitial(excel_path)
    
    workbook = load_workbook(excel_path)
    sheet_names = workbook.sheetnames   
    # Iterate over all sheets and call Fill_HalfLife_Sheet() if the sheet name includes "MeV"
    for sheet_name in sheet_names:
        if "MeV" in sheet_name:
            df = sheets[sheet_name]
            df = Cal_Prodx_Rate(df, thick_tot, mass, rate)
            # Save df back to Excel
            with pd.ExcelWriter(excel_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
                df.to_excel(writer, sheet_name=sheet_name, index=False)
            # Reopen the workbook to apply formatting
            workbook = load_workbook(excel_path)
            # Set column width and alignment for the new sheet
            worksheet = workbook[sheet_name]
            for column in worksheet.columns:
                column_letter = column[0].column_letter
                worksheet.column_dimensions[column_letter].width = 16
                for cell in column:  # Align each cell in the column
                    cell.alignment = Alignment(horizontal='right')
            # Save the workbook
            workbook.save(excel_path)

In [36]:
excel_path = '/Users/yiyizhu/Packages/DoesCal/example/DoesCal.xlsx'
Write_Prodx_Rate(excel_path)